In [ ]:
# 统计长度
from collections import Counter
Counter(list(df_train_test.groupby(['flag','fragment_id','behavior_id']).size()))

In [ ]:
seq_len = 64#这里设置64是为了方便以后的编码
def agg_func(x):#如果不满就填充0
    list_x = list(x)
    len_x = len(list_x)
    if len_x <= seq_len:
        list_x = [0] * (seq_len-len_x) + list_x
    else:
        list_x = list_x[:seq_len]
    return list_x

map_agg_func = {    
    'time_point' : agg_func,
    
    'acc_all' : agg_func,
    'acc_allg' : agg_func,
    
    'acc_x' : agg_func,
    'acc_y' : agg_func,
    'acc_z' : agg_func,
    
    'acc_xg' : agg_func,
    'acc_yg' : agg_func,
    'acc_zg' : agg_func
}

In [ ]:
df_train_test_list = df_train_test.groupby(['flag','fragment_id','behavior_id']).agg(map_agg_func).reset_index()

In [ ]:
df_train_test_list.drop(['flag','fragment_id','behavior_id'],axis=1,inplace=True)

In [ ]:
list_features = []
for index, row in tqdm(df_train_test_list.iterrows()):
    acc_all = np.array(row['acc_all'])
    acc_allg = np.array(row['acc_allg'])
    acc_x = np.array(row['acc_x'])
    acc_y = np.array(row['acc_y'])
    acc_z = np.array(row['acc_z'])
    acc_xg = np.array(row['acc_xg'])
    acc_yg = np.array(row['acc_yg'])
    acc_zg = np.array(row['acc_zg'])
    
    features = np.stack([acc_all,acc_allg,acc_x,acc_y,acc_z,acc_xg,acc_yg,acc_zg]).T
    list_features.append(features)#这个循环把每一行里每一列的list提出出来，然后stack之后转置，再加入到一个新的list里，这个list是三维的
np.array(list_features).shape

In [ ]:
import torch
ok= torch.from_numpy(np.array(list_features).reshape(8,14792,64))
ok1 = ok.float()#转化好格式

In [ ]:
encoder_layer = nn.TransformerEncoderLayer(d_model=64,nhead=4)
transformer_encoder=nn.TransformerEncoder(encoder_layer,num_layers=6)
out =transformer_encoder(ok1)#把一串64长度的序列转化为一个64维的向量，增强他的表示能力？ 

In [ ]:
pd.DataFrame(out.detach().numpy().reshape(14792,512))
#最后再把输出的格式转化成DF格式与原始数据拼接？